# Exercise: practice SQLAlchemy 🧪

Let's configure our first RDS instance on AWS and then use SQLAlchemy to write into our remote database and make some SQL queries!

1. Follow the instructions/videos from yesterday (M03-D03/04-Amazon-RDS.ipynb) to create your own RDS instance on AWS 
2. Download <a href="https://www.pgadmin.org/download/" target="_blank">PGAdmin</a> and configure it to access your remote database

If you get stuck at some step, don't hesitate to ask for help to your classmates, your teacher or your TA 🤗.

**Before continuing, please execute the cell below, this will install a package that is required to access your remote database from this notebook:**

In [1]:
# ! pip install psycopg2-binary
# ! pip install python-dotenv

3. Create an sqlalchemy engine that is connected to your AWS RDS instance

In [2]:
# Import sqlalchemy
from sqlalchemy import create_engine, text
import pandas as pd
import plotly.express as px

from dotenv import load_dotenv
import os

load_dotenv()
%load_ext dotenv
%dotenv

DBUSERNAME = os.getenv('DBUSERNAME')
DBPASSWORD = os.getenv('DBPASSWORD')
DBHOSTNAME = os.getenv('DBHOSTNAME')
DBDBNAME = os.getenv('DBNAME')


# Create engine will create a connection between a SQLlite DB and python
# engine = create_engine("sqlite:///:memory:", echo=True)
# engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}", echo=True)
# engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)
engine = create_engine(f"postgresql+psycopg2://{DBUSERNAME}:{DBPASSWORD}@{DBHOSTNAME}/{DBDBNAME}", echo=True)

In [3]:
# Let's instanciate a declarative base to be able to use our python class
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Let's define our table using a class
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "customers"

    # Each parameter corresponds to a column in our DB table
    id = Column(Integer, primary_key=True)
    name = Column(String)
    country = Column(String)
    job = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return "<User(name='{}', country='{}', job='{}', age='{}')>".format(self.name, self.country, self.job, self.age)

4. Create a new table named `customers` in your remote database and insert the following data :

| id | name       | country        | job        | age |
|----|------------|----------------|------------|-----|
| 1  | Sauerkraut | Germany        | engineer   | 37  |
| 2  | Jones      | United Kingdom | journalist | 52  |
| 3  | Dupont     | France         | dancer     | 25  |

Optionnal: Use PGAdmin to check that the table has been created without any mistake

In [4]:
Base.metadata.create_all(engine)

2022-04-04 15:28:04,992 INFO sqlalchemy.engine.Engine select version()
2022-04-04 15:28:04,993 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-04 15:28:05,169 INFO sqlalchemy.engine.Engine select current_schema()
2022-04-04 15:28:05,170 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-04 15:28:05,345 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-04-04 15:28:05,346 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-04 15:28:05,523 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-04 15:28:05,525 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:28:05,526 INFO sqlalchemy.engine.Engine [generated in 0.00116s] {'name': 'customers'}
2022-04-04 15:28:05,702 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# Create instances of User

# Create a new instance of User will allow us to insert a new record later on
ed_user = User(id=1, name='Berenger Queune', country='France', job='Data Engineer', age=68)

# Access Full row 
print(ed_user)

# Access ed_user name 
name = ed_user.name
print("name: {}".format(name))

# Access ed_user nickname
nickname = ed_user.job 
print("nickname: {}".format(nickname))

# Initialize a sessionmaker 
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

# Instanciate Session 

session = Session()

# Add values to db 

al_user = User(id=2, name='Hajime No Ippo', country='Japon', job='Boxeur', age=20)

session.add(ed_user)
session.add(al_user)
# Commit the results 

session.commit()

In [7]:
session = Session()


# Query our table users
user = session.query(User)

# Output all the results 
user.all()

2022-04-04 15:30:52,541 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-04 15:30:52,544 INFO sqlalchemy.engine.Engine SELECT customers.id AS customers_id, customers.name AS customers_name, customers.country AS customers_country, customers.job AS customers_job, customers.age AS customers_age 
FROM customers
2022-04-04 15:30:52,545 INFO sqlalchemy.engine.Engine [generated in 0.00118s] {}


[<User(name='Berenger Queune', country='France', job='Data Engineer', age='68')>,
 <User(name='Hajime No Ippo', country='Japon', job='Boxeur', age='20')>]

5. Execute the cell below to download the famous iris dataset:

In [19]:
data = px.data.iris()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


6. Create a table in your remote database containing the information of the dataset:

In [ ]:
data.to_sql(
    "iris",
    engine
)

### Now let's make some SQL requests !

To answer the following questions, don't hesitate to refer to <a href="https://www.sqltutorial.org/sql-cheat-sheet/" target="_blank">this cheatsheet</a>. 😉

7. What are the different species present in this dataset?

In [21]:
species = text("SELECT DISTINCT species "
             "FROM iris")

pd.read_sql(species, engine)

2022-04-04 15:42:59,990 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:42:59,991 INFO sqlalchemy.engine.Engine [cached since 894.5s ago] {'name': 'SELECT DISTINCT species FROM iris'}
2022-04-04 15:43:00,255 INFO sqlalchemy.engine.Engine SELECT DISTINCT species FROM iris
2022-04-04 15:43:00,255 INFO sqlalchemy.engine.Engine [cached since 40.71s ago] {}


,species
0,setosa
1,virginica
2,versicolor


8. What is the average sepal length among all species?

In [22]:
length = text("SELECT AVG(sepal_length) "
             "FROM iris")

pd.read_sql(length, engine)

2022-04-04 15:44:45,439 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:44:45,441 INFO sqlalchemy.engine.Engine [cached since 999.9s ago] {'name': 'SELECT AVG(sepal_length) FROM iris'}
2022-04-04 15:44:45,707 INFO sqlalchemy.engine.Engine SELECT AVG(sepal_length) FROM iris
2022-04-04 15:44:45,707 INFO sqlalchemy.engine.Engine [generated in 0.00086s] {}


,avg
0,5.843333


9. What is the average sepal length for each species?

In [23]:
average_by_species = text("SELECT species, AVG(sepal_length) "
             "FROM iris "
             "GROUP BY species")

pd.read_sql(average_by_species, engine)

2022-04-04 15:48:34,800 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:48:34,802 INFO sqlalchemy.engine.Engine [cached since 1229s ago] {'name': 'SELECT species, AVG(sepal_length) FROM iris GROUP BY species'}
2022-04-04 15:48:35,068 INFO sqlalchemy.engine.Engine SELECT species, AVG(sepal_length) FROM iris GROUP BY species
2022-04-04 15:48:35,068 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {}


,species,avg
0,setosa,5.006
1,virginica,6.588
2,versicolor,5.936


10. How many samples of 'virginica' have sepal length < 6?

In [24]:
virginica_6 = text("SELECT COUNT(*) "
             "FROM iris "
             "WHERE species = 'virginica' "
             "AND sepal_length < 6")

pd.read_sql(virginica_6, engine)

2022-04-04 15:49:16,080 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:49:16,082 INFO sqlalchemy.engine.Engine [cached since 1271s ago] {'name': "SELECT COUNT(*) FROM iris WHERE species = 'virginica' AND sepal_length < 6"}
2022-04-04 15:49:16,346 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM iris WHERE species = 'virginica' AND sepal_length < 6
2022-04-04 15:49:16,346 INFO sqlalchemy.engine.Engine [generated in 0.00080s] {}


,count
0,7


11. For each species, count the number of samples having sepal length < 6:

In [29]:
all_species_6 = text("SELECT species, COUNT(*) "
             "FROM iris "
             "WHERE sepal_length < 6 "
             "GROUP BY species")

pd.read_sql(all_species_6, engine)

2022-04-04 15:51:34,338 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-04 15:51:34,339 INFO sqlalchemy.engine.Engine [cached since 1409s ago] {'name': 'SELECT species, COUNT(*) FROM iris WHERE sepal_length < 6 GROUP BY species'}
2022-04-04 15:51:34,602 INFO sqlalchemy.engine.Engine SELECT species, COUNT(*) FROM iris WHERE sepal_length < 6 GROUP BY species
2022-04-04 15:51:34,603 INFO sqlalchemy.engine.Engine [generated in 0.00076s] {}


,species,count
0,setosa,50
1,virginica,7
2,versicolor,26


In [31]:
# test of stmt engine connection

conn = engine.connect()

stmt = text("SELECT country FROM customers")
result = conn.execute(stmt)
result.fetchall()

2022-04-04 15:57:19,490 INFO sqlalchemy.engine.Engine SELECT country FROM customers
2022-04-04 15:57:19,491 INFO sqlalchemy.engine.Engine [generated in 0.00077s] {}


[('France',), ('Japon',)]